**Figure Architecture**

In [41]:
root = "../data/plots/final"

import fitz  # PyMuPDF
from io import BytesIO


def merge_vertically(pdf1_path, pdf2_path, output_path, padding=35, top_padding=20):
    TARGET_WIDTH_PT = 510  # 180 mm in points

    doc1 = fitz.open(pdf1_path)
    doc2 = fitz.open(pdf2_path)

    page1 = doc1[0]
    page2 = doc2[0]

    # Get original sizes
    fig1_rect = page1.rect
    fig2_rect = page2.rect

    # Calculate scale factors to make both widths 510 pt
    scale1 = TARGET_WIDTH_PT / fig1_rect.width
    scale2 = TARGET_WIDTH_PT / fig2_rect.width

    # Calculate new heights based on scale
    fig1_scaled_height = fig1_rect.height * scale1
    fig2_scaled_height = fig2_rect.height * scale2

    output_width = TARGET_WIDTH_PT
    output_height = top_padding + fig1_scaled_height + padding + fig2_scaled_height

    merged = fitz.open()
    new_page = merged.new_page(width=output_width, height=output_height)
    font_bold = fitz.Font(fontbuffer=bold_font_buffer)
    # font_bold = new_page.insert_font(fontbuffer=bold_font_buffer, name="bold-font")

    # Define placement rectangles (x=0 for full width)
    rect1 = fitz.Rect(0, top_padding, TARGET_WIDTH_PT, top_padding + fig1_scaled_height)
    rect2 = fitz.Rect(0, top_padding + fig1_scaled_height + padding,
                      TARGET_WIDTH_PT, output_height)


    # Borders
    # shape = new_page.new_shape()
    # shape.draw_rect(rect1)
    # shape.draw_rect(rect2)
    # shape.finish(width=0.05, color=(0.8, 0.8, 0.8))
    # shape.commit()

    
    # Show scaled figures
    new_page.show_pdf_page(rect1, doc1, 0, clip=fig1_rect)
    new_page.show_pdf_page(rect2, doc2, 0, clip=fig2_rect)

    

    # Labels
    font_size = 10
    label_padding = 0

    # A: bold "A:", regular "Deep architecture"
    label_a_x = rect1.x0 + label_padding
    label_a_y = rect1.y0 + label_padding - 10
    new_page.insert_text((label_a_x, label_a_y),
                         "A:", fontsize=font_size, fontname="helvetica-bold", fill=(0, 0, 0))
    a_width = font_bold.text_length("A:", fontsize=font_size)
    new_page.insert_text((label_a_x + a_width, label_a_y),
                         " Deep architecture", fontsize=font_size, fontname="helvetica", fill=(0, 0, 0))
    
    # B: bold "B:", regular "Localized architecture"
    label_b_x = rect2.x0 + label_padding
    label_b_y = rect2.y0 + label_padding - 10
    new_page.insert_text((label_b_x, label_b_y),
                         "B:", fontsize=font_size, fontname="helvetica-bold", fill=(0, 0, 0))
    b_width = font_bold.text_length("B:", fontsize=font_size)
    new_page.insert_text((label_b_x + b_width, label_b_y),
                         " Localized architecture", fontsize=font_size, fontname="helvetica", fill=(0, 0, 0))

    # Save
    merged.save(output_path)
    merged.close()
    doc1.close()
    doc2.close()

# Usage
path1 = f'{root}/DeepSkip.pdf'
path2 = f'{root}/Local.pdf'
path3 = f'{root}/Architecture.pdf'

merge_vertically(path1, path2, path3)


**Figure Long-term**

In [44]:
import fitz  # PyMuPDF

def merge_vertically(pdf1_path, pdf2_path, pdf3_path, output_path, padding=35, top_padding=20):
    TARGET_WIDTH_PT = 510  # 180 mm in points
    width2_ratio = 0.75
    width3_ratio = 0.25

    doc1 = fitz.open(pdf1_path)
    doc2 = fitz.open(pdf2_path)
    doc3 = fitz.open(pdf3_path)

    page1 = doc1[0]
    page2 = doc2[0]
    page3 = doc3[0]

    fig1_rect = page1.rect
    fig2_rect = page2.rect
    fig3_rect = page3.rect

    # Scale Figure 1 to full width
    scale1 = TARGET_WIDTH_PT / fig1_rect.width
    h1 = fig1_rect.height * scale1

    # Set target widths
    width2 = width2_ratio * TARGET_WIDTH_PT
    width3 = width3_ratio * TARGET_WIDTH_PT

    # Scale both Figures 2 and 3 to same height (row_height)
    scale2 = width2 / fig2_rect.width
    scale3 = width3 / fig3_rect.width

    h2 = fig2_rect.height * scale2
    h3 = fig3_rect.height * scale2

    # Set common height for both figures (min of the two to avoid overflow)
    row_height = min(h2, h3)

    # Recompute scales to match row_height
    scale2 = row_height / fig2_rect.height
    scale3 = row_height / fig3_rect.height
    width2 = fig2_rect.width * scale2
    width3 = fig3_rect.width * scale3

    # Compute total height of the merged page
    output_height = top_padding + h1 + padding + row_height + padding

    merged = fitz.open()
    new_page = merged.new_page(width=TARGET_WIDTH_PT, height=output_height)

    # Placement rects
    rect1 = fitz.Rect(0, top_padding, TARGET_WIDTH_PT, top_padding + h1)

    y_row = rect1.y1 + padding
    rect2 = fitz.Rect(0, y_row, width2, y_row + row_height)
    rect3 = fitz.Rect(width2, y_row, width2 + width3, y_row + row_height)

    # Insert figures
    new_page.show_pdf_page(rect1, doc1, 0, clip=fig1_rect)
    new_page.show_pdf_page(rect2, doc2, 0, clip=fig2_rect)
    new_page.show_pdf_page(rect3, doc3, 0, clip=fig3_rect)

    # Labels
    font_size = 10
    label_offset_y = 12
    label_padding = 0

    # A: top
    new_page.insert_text((rect1.x0 + label_padding + 10, rect1.y0 - label_offset_y), "A:", fontsize=font_size, fontname="helvetica-bold")
    new_page.insert_text((rect1.x0 + label_padding + 20, rect1.y0 - label_offset_y), " Lorenz-63", fontsize=font_size, fontname="helvetica")

    # B: bottom left
    new_page.insert_text((rect2.x0 + label_padding + 10, rect2.y0 - label_offset_y), "B:", fontsize=font_size, fontname="helvetica-bold")
    new_page.insert_text((rect2.x0 + label_padding + 20, rect2.y0 - label_offset_y), " Lorenz-96", fontsize=font_size, fontname="helvetica")

    # C: bottom right
    new_page.insert_text((rect3.x0 + label_padding + 10, rect2.y0 - label_offset_y), "C:", fontsize=font_size, fontname="helvetica-bold")
    new_page.insert_text((rect3.x0 + label_padding + 20, rect2.y0 - label_offset_y), " Kuramoto-Sivashinsky", fontsize=font_size, fontname="helvetica")

    # Save and close
    merged.save(output_path)
    merged.close()
    doc1.close()
    doc2.close()
    doc3.close()

# Usage
path1 = f'{root}/L63-climate.pdf'
path2 = f'{root}/L96-climate.pdf'
path3 = f'{root}/KS-200-climate.pdf'
path4 = f'{root}/Long-term.pdf'

merge_vertically(path1, path2, path3, path4)


**Figure: short-term**

In [43]:
import fitz  # PyMuPDF

def merge_vertically(pdf1_path, pdf2_path, pdf3_path, output_path, padding=25, top_padding=20):
    TARGET_WIDTH_PT = 510  # 180 mm in points

    doc1 = fitz.open(pdf1_path)
    doc2 = fitz.open(pdf2_path)
    doc3 = fitz.open(pdf3_path)

    page1 = doc1[0]
    page2 = doc2[0]
    page3 = doc3[0]

    fig1_rect = page1.rect
    fig2_rect = page2.rect
    fig3_rect = page3.rect

    # Compute the maximum original width
    max_original_width = max(fig1_rect.width, fig2_rect.width, fig3_rect.width)

    # Compute a uniform scale factor so the widest figure fits the target width
    scale = TARGET_WIDTH_PT / max_original_width

    # Compute scaled heights
    h1 = fig1_rect.height * scale
    h2 = fig2_rect.height * scale
    h3 = fig3_rect.height * scale

    # Compute scaled widths (for centered placement)
    w1 = fig1_rect.width * scale
    w2 = fig2_rect.width * scale
    w3 = fig3_rect.width * scale

    # Compute output page height
    output_height = top_padding + h1 + h2 + h3 + 2 * padding

    merged = fitz.open()
    new_page = merged.new_page(width=TARGET_WIDTH_PT, height=output_height)

    # Placement rectangles (centered horizontally)
    rect1 = fitz.Rect((TARGET_WIDTH_PT - w1) / 2, top_padding,
                      (TARGET_WIDTH_PT + w1) / 2, top_padding + h1)
    rect2 = fitz.Rect((TARGET_WIDTH_PT - w2) / 2, rect1.y1 + padding,
                      (TARGET_WIDTH_PT + w2) / 2, rect1.y1 + padding + h2)
    rect3 = fitz.Rect((TARGET_WIDTH_PT - w3) / 2, rect2.y1 + padding,
                      (TARGET_WIDTH_PT + w3) / 2, rect2.y1 + padding + h3)

    # Insert pages
    new_page.show_pdf_page(rect1, doc1, 0, clip=fig1_rect)
    new_page.show_pdf_page(rect2, doc2, 0, clip=fig2_rect)
    new_page.show_pdf_page(rect3, doc3, 0, clip=fig3_rect)

    # Labels
    font_size = 10
    label_padding = 10

    new_page.insert_text((rect1.x0 + label_padding, rect1.y0 + label_padding - 10),
                         "A: Lorenz-63", fontsize=font_size, fontname="helv", fill=(0, 0, 0))
    new_page.insert_text((rect2.x0 + label_padding, rect2.y0 + label_padding - 10),
                         "B: Lorenz-96", fontsize=font_size, fontname="helv", fill=(0, 0, 0))
    new_page.insert_text((rect3.x0 + label_padding, rect3.y0 + label_padding - 10),
                         "C: Kuramoto-Sivashinsky", fontsize=font_size, fontname="helv", fill=(0, 0, 0))

    merged.save(output_path)
    merged.close()
    doc1.close()
    doc2.close()
    doc3.close()

# Usage
path1 = f'{root}/L63-config_1_s-vpt-violin.pdf'
path2 = f'{root}/L96-config_1_s-vpt-violin.pdf'
path3 = f'{root}/KS-200-config_1-vpt-violin.pdf'
path4 = f'{root}/Short-term.pdf'

merge_vertically(path1, path2, path3, path4)